**Objectives:**

Calculate the customer lifetime value for different segments of customers.

Identify the most valuable customer segments.

Develop personalized marketing strategies to increase customer retention and maximize CLV.

Lógica:
Importar base  
Visualizar base  
Tratar base removendo duplicatas e valores vazios  
Tratar a questão de datas  
Criar colunas necessárias para calculo de CLV  
Descobrir o CLV  
Criar segmentações a partir do CLV  
Desenvolver estratégias de marekting para reter e maximizar o cliente.  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from datetime import date

# Visualizando dataframe
# (81601, 10)

   #Unnamed: 0   81601 non-null  int64
   #InvoiceNo    81601 non-null  object
    #StockCode    81601 non-null  object
    #Description  81601 non-null  object
    #Quantity     81601 non-null  int64
    #UnitPrice    81601 non-null  float64
    #CustomerID   81601 non-null  float64
    #Country      81601 non-null  object
    #Date         81601 non-null  object
    #Hour         81601 non-null  int64

# Colunas: InvoiceNo - StockCode - Description - Quantity - UnitPrice - CustomerID - Country - Date - Hour

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CRM/ABC Electronics/Ecommerce_Data-1.csv')
df = df[df['Quantity']>0]
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour
0,439570,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12
1,387281,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10
2,337863,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9
3,57628,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329.0,United Kingdom,2011-01-14,13
4,330897,565930,POST,POSTAGE,5,18.00,12685.0,France,2011-09-08,10


In [3]:
#Tratando os dados

df1 = df.drop(columns="Unnamed: 0")
df1['CustomerID'] = df1['CustomerID'].astype(int)

#Convertendo Date para dtype=Datetime
df1['Date'] = pd.to_datetime(df['Date'])

df1 = df1.dropna()

df1['Total_Revenue'] = df1['Quantity']*df1['UnitPrice']

df1

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour,Total_Revenue
0,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453,United Kingdom,2011-11-04,12,3.25
1,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098,United Kingdom,2011-10-10,10,89.40
2,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609,United Kingdom,2011-09-13,9,45.00
3,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329,United Kingdom,2011-01-14,13,16.50
4,565930,POST,POSTAGE,5,18.00,12685,France,2011-09-08,10,90.00
...,...,...,...,...,...,...,...,...,...,...
81596,560692,21294,ETCHED GLASS COASTER,4,0.39,14704,United Kingdom,2011-07-20,12,1.56
81597,540187,22569,FELTCRAFT CUSHION BUTTERFLY,4,3.75,15358,United Kingdom,2011-01-05,13,15.00
81598,547871,17012C,ORIGAMI LAVENDER INCENSE/CANDL SET,5,2.55,16931,United Kingdom,2011-03-27,14,12.75
81599,562129,22138,BAKING SET 9 PIECE RETROSPOT,3,4.95,13186,United Kingdom,2011-08-03,9,14.85


In [47]:
#Segregando por clientes cada informação necessária para calcular seu CLV

df2 = df1.groupby('CustomerID').agg(
    Last_Sale =('Date', 'max'),
    LifeCycle =('Date', lambda x: x.max() - x.min()),
    Quantity = ('Quantity', 'sum'),
    Total_Revenue = ('Total_Revenue', 'sum'),
    Orders = ('InvoiceNo', 'nunique'))

margem_lucro = 50/100

df2['LifeCycle'] = df2['LifeCycle'].dt.days.astype('int32')

df2['Average_Ticket'] = df2['Total_Revenue']/df2['Orders']
df2['Average_Orders_Period']= df2['Orders']/df2['LifeCycle']
df2['CLV'] = df2['Average_Ticket']*df2['LifeCycle']*df2['Average_Orders_Period']*margem_lucro

#df2['CLV'].describe()

#4094 Clientes no dataframe
#df2.shape

df2

,Last_Sale,LifeCycle,Quantity,Total_Revenue,Orders,Average_Ticket,Average_Orders_Period,CLV
CustomerID,,,,,,,,
12347,2011-12-07,365,623,1146.59,7,163.798571,0.019178,573.295
12348,2011-09-25,283,531,550.92,4,137.730000,0.014134,275.460
12349,2011-11-21,0,203,406.32,1,406.320000,inf,NaN
12350,2011-02-02,0,36,45.60,1,45.600000,inf,NaN
12352,2011-11-03,260,163,663.33,6,110.555000,0.023077,331.665
...,...,...,...,...,...,...,...,...
18280,2011-03-07,0,9,52.75,1,52.750000,inf,NaN
18281,2011-06-12,0,11,33.45,1,33.450000,inf,NaN
18282,2011-12-02,119,11,67.85,2,33.925000,0.016807,33.925


In [9]:
df3 = df2.sort_values(by='CLV',ascending=0)

#Temos 1579 clientes da base que fizeram apenas uma compra ou 38,5% da base
#Celula Vazia = null
compra = df3['CLV'].isna().sum()/4094
compra

0.38568637029799707

In [17]:
#Tratando NaN e inf

df2['CLV'] = df2['CLV']. fillna(0)

def correcao_average_orders(valor):
  if(valor) == float('inf'):
      return (0)
  else:
      return (valor)

df2['Average_Orders_Period'] = df2['Average_Orders_Period'].apply(correcao_average_orders)

df2.describe()


,LifeCycle,Quantity,Total_Revenue,Orders,Average_Ticket,Average_Orders_Period,CLV
count,4094.000000,4094.000000,4094.000000,4094.000000,4094.000000,4094.000000,4094.000000
mean,124.932096,244.896190,417.610215,3.777723,92.620074,0.032579,191.191351
std,131.370426,978.071326,1666.980624,6.437994,198.190511,0.396219,834.144300
min,0.000000,1.000000,0.780000,1.000000,0.780000,0.000000,0.000000
25%,0.000000,32.000000,62.640000,1.000000,38.285000,0.000000,0.000000
50%,79.000000,81.000000,143.950000,2.000000,63.375000,0.013605,56.212500
75%,245.750000,211.000000,358.907500,4.000000,103.673125,0.027027,168.922500
max,373.000000,38077.000000,51732.280000,183.000000,10356.650000,24.000000,25866.140000


In [18]:
def status_cliente(valor):
    if pd.notna(valor):  # Verifica se o valor não é NaN
        if valor <= 0 :
            return 'Primeira Compra'
        elif valor <= 136/2:
            return 'Baixo Retorno - Micro Pulverizado'
        elif valor <= 600/2:
            return 'Medio Retorno - Pulverizado'
        elif valor <= 1500/2:
            return 'Medio/Alto Retorno - Pulverizado'
        elif valor <= 10000/2:
            return 'Alto Retorno - Grandes Clientes B'
        elif valor <= 30000/2:
            return 'Grandes Clientes A - Grandes Clientes A'
        else:
            return 'Valor acima de 25000 - Categoria Especial'


df2['Status'] = df2['CLV'].apply(status_cliente)

df2['Status'].value_counts()

Primeira Compra                              1579
Medio Retorno - Pulverizado                  1338
Baixo Retorno - Micro Pulverizado             629
Medio/Alto Retorno - Pulverizado              404
Alto Retorno - Grandes Clientes B             129
Grandes Clientes A - Grandes Clientes A        12
Valor acima de 25000 - Categoria Especial       3
Name: Status, dtype: int64

In [ ]:
df2.to_csv('/content/drive/MyDrive/Colab Notebooks/CRM/ABC Electronics/crm_tratado.csv')

In [43]:
df2

,Last_Sale,LifeCycle,Quantity,Total_Revenue,Orders,Average_Ticket,Average_Orders_Period,CLV,Status
CustomerID,,,,,,,,,
12347,2011-12-07,365,623,1146.59,7,163.798571,0.019178,573.295,Medio/Alto Retorno - Pulverizado
12348,2011-09-25,283,531,550.92,4,137.730000,0.014134,275.460,Medio Retorno - Pulverizado
12349,2011-11-21,0,203,406.32,1,406.320000,0.000000,0.000,Primeira Compra
12350,2011-02-02,0,36,45.60,1,45.600000,0.000000,0.000,Primeira Compra
12352,2011-11-03,260,163,663.33,6,110.555000,0.023077,331.665,Medio/Alto Retorno - Pulverizado
...,...,...,...,...,...,...,...,...,...
18280,2011-03-07,0,9,52.75,1,52.750000,0.000000,0.000,Primeira Compra
18281,2011-06-12,0,11,33.45,1,33.450000,0.000000,0.000,Primeira Compra
18282,2011-12-02,119,11,67.85,2,33.925000,0.016807,33.925,Baixo Retorno - Micro Pulverizado


Infos Importantes:
4094 Clientes
1.709.696 Faturamento total


Ações Possíveis:  
Primeira Compra: Rentabilizar os clientes que fizeram apenas uma compra na empresa e não retornaram (corresponde a cerca de 39% da base de clientes)  
  
  
Micropulverizado: Lembretes mensais/bimestrais para realizar uma próxima compra  
  
  
Medio/Alto Retorno: Reter oferecendo vantagens e comunicações exclusivas, sendo eventos e outras participações (Esse segmento corresponde a 21% do faturamento total da empresa)  
  
  
Alto Retorno: Distribuir gerentes de contas executivos para tratar com o cliente, tendo reuniões mensais ou quinzenais (a depender do nível de procura do produto)  


Grandes/Categoria Especial: Tratar com negociações com Diretor Executivo da empresa. Por ter uma alta demanda, é possível oferecer sistema de crédito e outras vantagens por pedido.  
(esse tipo de cliente possui um padrão por pedido superior a 10.000 unidades)   
(Existem 3 clientes Especiais e 12 Clientes A ; representa 20% do faturamento total)  

In [54]:
faturamento_medioalto = (362123)/(1709696)
faturamento_medioalto

faturamento_a_especial = (132729+198079)/1709696

faturamento_a_especial

0.1934893688702553